In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
pip install efficientnet

     |████████████████████████████████| 51kB 4.0MB/s 


In [ ]:
import datetime
import os
import numpy as np
import tensorflow as tf

from keras import backend as K
from efficientnet.keras import EfficientNetB7
from tensorflow.keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Conv2DTranspose
from tensorflow.keras.layers import Dropout, LeakyReLU, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [ ]:
np.random.seed = 42

dataset = "DRIVE"
dim = 128 if dataset == "DRIVE" else 192

main_path = f"/content/gdrive/My Drive/Data/{dataset}"

In [ ]:
training_images = f"{main_path}/patches_{dim}/images/train/"
training_labels = f"{main_path}/patches_{dim}/labels/train/"

train_img = next(os.walk(training_images))[2]
train_lbs = next(os.walk(training_labels))[2]

train_img.sort()
train_lbs.sort()

In [ ]:
X_train = np.concatenate([np.load(training_images + file_id)["arr_0"] for file_id in train_img], axis=0)
Y_train = np.concatenate([np.load(training_labels + file_id)["arr_0"] for file_id in train_lbs], axis=0)

X_train = X_train / 255

if dataset == "CHASEDB1":
    Y_train = Y_train.astype("float64")
else:
    Y_train = Y_train / 255

In [ ]:
print(np.max(X_train))
print(np.min(X_train))
print(np.max(Y_train))
print(np.min(Y_train))

1.0
0.0
1.0
0.0


In [ ]:
X_train.shape

(3200, 128, 128, 3)

In [ ]:
_, h, w, c = X_train.shape

input_shape = (h, w, c)

In [ ]:
def Conv_block(x, filters, a=0.01, dr=0.05):
    y = Conv2D(filters, (3, 3), strides=(1, 1), padding="same")(x)
    y = BatchNormalization(axis=-1)(y)
    y = LeakyReLU(alpha=a)(y)
    y = Dropout(rate=dr)(y)

    return y

In [ ]:
def Dense_block(x, filters, a=0.01, dr=0.05):
    x1 = Conv_block(x, filters, a, dr)
    c1 = Concatenate(axis=-1)([x, x1])

    x2 = Conv_block(c1, filters, a, dr)
    c2 = Concatenate(axis=-1)([c1, x2])

    x3 = Conv_block(c2, filters, a, dr)
    c3 = Concatenate(axis=-1)([c2, x3])

    return c3

In [ ]:
def efficientB7DenseUnet(input_shape):
    a = 0.01
    dr = 0.05

    Encoder = EfficientNetB7(include_top=False, weights="imagenet", input_shape=input_shape)

    # contracting level 1
    input = Encoder.input
    en1 = Dense_block(input, 48, a, dr)

    # contracting level 2
    en2 = Encoder.layers[49].output

    # contracting level 3
    en3 = Encoder.layers[152].output

    # contracting level 4
    en4 = Encoder.layers[255].output

    # middle level 5
    en5 = Encoder.layers[551].output
    cv5 = Dense_block(en5, 160, a, dr)
    up5 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding="same")(cv5)

    # expandsion level 4
    cv4 = Concatenate(axis=-1)([up5, en4])
    cv4 = Dense_block(cv4, 128, a, dr)
    up4 = Conv2DTranspose(96, (2, 2), strides=(2, 2), padding="same")(cv4)

    # expandsion level 3
    cv3 = Concatenate(axis=-1)([up4, en3])
    cv3 = Dense_block(cv3, 96, a, dr)
    up3 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(cv3)

    # expandsion level 2
    cv2 = Concatenate(axis=-1)([up3, en2])
    cv2 = Dense_block(cv2, 64, a, dr)
    up2 = Conv2DTranspose(48, (2, 2), strides=(2, 2), padding="same")(cv2)

    # expandsion level 1
    cv1 = Concatenate(axis=-1)([up2, en1])
    cv1 = Dense_block(cv1, 48, a, dr)

    cv1 = Conv2D(48, (3, 3), activation="relu", padding="same")(cv1)

    out = Conv2D(1, (1, 1), activation="sigmoid")(cv1)

    model = tf.keras.Model(inputs=[input], outputs=[out])

    # model.compile(optimizer=Adam(2e-4), loss=dice_coef_loss, metrics=[dice_coef, "acc", "mse"])
    model.compile(optimizer=Adam(0.001), loss="binary_crossentropy", metrics=["acc", "mse"])

    return model

In [ ]:
B = 7
blocks = "DenseUnet"

model_name = f"EffnetB{B}_{blocks}"

In [ ]:
# model = efficientB7DenseUnet(input_shape)

In [ ]:
model = tf.keras.models.load_model(f"/content/gdrive/My Drive/NN Models/{dataset}/model_DRIVE_EffnetB7_DenseUnet_2021-02-12 17:16:11")

In [ ]:
# K.set_value(model.optimizer.learning_rate, 0.0003)
# K.set_value(model.optimizer.learning_rate, 0.0001)
# K.set_value(model.optimizer.learning_rate, 0.00003)
# K.set_value(model.optimizer.learning_rate, 0.00001)
# K.set_value(model.optimizer.learning_rate, 0.000003)
# K.set_value(model.optimizer.learning_rate, 0.000001)

In [ ]:
# model.summary()

In [ ]:
model.fit(x=X_train, y=Y_train, batch_size=16, epochs=50, verbose=2)

Epoch 1/50
200/200 - 89s - loss: 0.0069 - acc: 0.9973 - mse: 0.0020
Epoch 2/50
200/200 - 66s - loss: 0.0069 - acc: 0.9973 - mse: 0.0020
Epoch 3/50
200/200 - 66s - loss: 0.0068 - acc: 0.9973 - mse: 0.0020
Epoch 4/50
200/200 - 66s - loss: 0.0069 - acc: 0.9973 - mse: 0.0020
Epoch 5/50
200/200 - 66s - loss: 0.0069 - acc: 0.9973 - mse: 0.0020
Epoch 6/50
200/200 - 66s - loss: 0.0069 - acc: 0.9973 - mse: 0.0020
Epoch 7/50
200/200 - 66s - loss: 0.0069 - acc: 0.9973 - mse: 0.0020
Epoch 8/50
200/200 - 66s - loss: 0.0070 - acc: 0.9973 - mse: 0.0020
Epoch 9/50
200/200 - 66s - loss: 0.0069 - acc: 0.9973 - mse: 0.0020
Epoch 10/50
200/200 - 66s - loss: 0.0070 - acc: 0.9973 - mse: 0.0020
Epoch 11/50
200/200 - 66s - loss: 0.0069 - acc: 0.9973 - mse: 0.0020
Epoch 12/50
200/200 - 66s - loss: 0.0069 - acc: 0.9973 - mse: 0.0020
Epoch 13/50
200/200 - 66s - loss: 0.0068 - acc: 0.9974 - mse: 0.0020
Epoch 14/50
200/200 - 66s - loss: 0.0069 - acc: 0.9973 - mse: 0.0020
Epoch 15/50
200/200 - 66s - loss: 0.0069 - 

KeyboardInterrupt: ignored

In [ ]:
model.save(f"/content/gdrive/My Drive/NN Models/{dataset}/model_{dataset}_{model_name}_{str(datetime.datetime.now())[:-7]}")

In [ ]:
print("\n-----------------------------------------------End of process-----------------------------------------------")